# Funkcje:

In [10]:
### WYMIANA KLUCZA DIFFIEGO-HELLMANA
def Diffie_Hellman_key_exchange(p,g,a,b):
    
    if is_prime(p) == True:
    
        if Mod(g,p).is_primitive_root() == True:
            
            x = power_mod(g,a,p)
            y = power_mod(g,b,p)
            s_k_a = power_mod(y,a,p)
            s_k_b = power_mod(x,b,p)
            s_k_a_b = power_mod(g,a*b,p)

            print("Użytkownicy uzgodnili liczbę pierwszą p = ",p)
            print("")
            print("Za jej pierwiastek pierwotny wzięli liczbę g = ",g)
            print("")
            print("Użytkownik A wybrał tajną liczbę :",a,", oraz przesłał użytkownikowi B wartość :",x)
            print("")
            print("Użytkownik B wybrał tajną liczbę :",b,", oraz przesłał użytkownikowi A wartość :",y)
            print("")
            print("Wspólnym tajnym kluczem obliczonym przez A jest :",s_k_a)
            print("")
            print("Wspólnym tajnym kluczem obliczonym przez B jest :",s_k_b)
            print("")
            print("Sprawdzamy zgodność kluczy :",s_k_a_b)
            
        else:
            print("Uzgodniona liczba g nie jest pierwiastkiem pierwotnym.")
    else:
        print("Wybrana liczba p nie jest pierwsza.")

### ALGORYTM GENEROWANIA KLUCZA ELGAMAL        
def ElGamal_public_key(p,g,k):
    
    if is_prime(p) == True:
        
        if Mod(g,p).is_primitive_root() == True:

            if k > 0 and k < p-1:
                x = power_mod(g,k,p)
                print("Użytkownik wybrał liczbę pierwszą: p = ",p)
                print("")
                print("Za jej pierwiastek pierwotny wziął liczbę: g = ",g)
                print("")
                print("Jego kluczem prywatnym jest liczba: k = ",k)
                print("")
                print("Klucz publiczny tego użytkownika to trójka:",(p,g,x))
                K = (p,g,x)
                
            else:
                print("Liczba wybrana przez użytkownika jest nieodpowiednia.")
        else:
            print("Uzgodniona liczba g nie jest pierwiastkiem pierwotnym.")
    else:
        print("Wybrana liczba p nie jest pierwsza.")
        
    return K

### FUNKCJE POMOCNICZE
def letters_to_indices(s):
    return [ord(char.upper()) - ord('A') for char in s if char.isalpha()]

def indices_to_letters(indices):
    return ''.join(chr(index + ord('A')) for index in indices)


### ALGORYTM SZYFROWANIA ELGAMAL
def ElGamal_encryption_algorithm(message,users_key,j):
    p = users_key[0]
    g = users_key[1]
    x = users_key[2]
    
    if j > 0 and j < p-1:
        M = letters_to_indices(message)

        C1 = power_mod(g,j,p)
        xj = power_mod(x,j,p)

        cipher = [(M[k]*xj)%p for k in range(len(M))]
        cipher_output = (C1,cipher)
        
        print("Użytkownik chce przesłać wiadomość do osoby o kluczu publicznym :",users_key)
        print("")
        print("Wiadomość którą chce wysłać to :",message)
        print("")
        print("Wybrał liczbę j = ",j)
        print("")
        print("Szyfrogramem jego wiadomośći jest :",cipher_output)
    else:
        print("Wybrana liczba j jest nieodpowiednia.")
    
    return cipher_output


### ALGORYTM DESZYFROWANIA ELGAMAL
def ElGamal_decryption_algorithm(cipher,secret_key,public_key):
    C1 = cipher[0]
    cipher_input = cipher[1]
    k = secret_key
    p = public_key[0]
    
    C1_rec = power_mod(C1,-k,p)
    decripted_cipher = [(cipher_input[k]*C1_rec)%p for k in range(len(cipher_input))]
    
    decrypted_message = indices_to_letters(decripted_cipher)
    
    print("Odszyfrowana wiadomość to:", decrypted_message)
    
    return decrypted_message


### METODA PRZELICZANIA
def Brute_force_method(a,b,n, details = False):
    if details == True:
        print("Rozważamy zagadnienie:",a,"^x =",b,"( mod",n,")\n")
    al = Mod(b,n)
    bl = Mod(a,n)
    i = 0
    y = Mod(1,n)
    while True:
        y = y*bl
        if y ==al:
            break
        i +=1
    if details == True:
        print("Wynik końcowy x =", i+1)
    return i+1

### ALGORYTM MAŁYCH I WIELKICH KROKÓW
def Baby_giant_step(a,b,n,details = False, sets_display=False):
    Zn = Integers(n)
    al = Zn(b)
    bl = Zn(a)
    m = ceil(sqrt(n))
    B = [al*bl^(-r) for r in range(m)]
    delta = bl^Integer(m)
    j = 0
    delq = Zn(1)
    delty = []
    if details == True:
        print("Rozważamy zagadnienie:",a,"^x =",b,"( mod",n,")\n")
        print("m =",m)
        print("")
    while j<m:
        j += 1
        delq = delq * delta
        delty.append(delq)
        for i in range(m):
            if delq==B[i]:
                x = m*j + i
                j=m
                break
    if details == True:
        if sets_display == True:
            print("Zbiór małych kroków:")
            print(B)
            print("")
            print("Zbiór wielkich kroków:")
            print(delty)
            print("")
        print("Pierwszy element zbioru wielkich kroków, który pokrył się z poprzednikiem w zbiorze małych kroków, to:",delq)
        print("")
        print("Ma on numer: j =",len(delty),", oraz odpowiada małemu krokowi z numerem: i =",B.index(delq))
        print("")
        print("Zgodnie ze wzorem, mamy: x =",len(delty),"*",m,"+",B.index(delq))
        print("")
        print("Wynik końcowy: x =",x)
    return x


### ALGORYTM POHLIGA-HELLMANA
def Pohlig_Hellman_algorithm(a,b,n, details = False):
    if details == True:
        print("Zagadnienie:",a,"^x =",b,"( mod",n,"), można zastąpić przez: \n")
    fa = factor(n-1)
    fl = len(fa)
    x=[]
    y=[]
    p_list=[]
    e_list=[]
    for i in range(fl):
        p=fa[i][0]
        e=fa[i][1]
        np = (n-1)/p^e
        bp = power_mod(b,np,n)
        ap = power_mod(a,np,n)
        
        disc = discrete_log(Mod(bp,n),Mod(ap,n))
        p_list.append(p)
        e_list.append(e)
        x.append(disc)
        y.append(fa[i][0]^fa[i][1])
        if details == True:
            print(ap,"^x(",p,") =",bp," (mod",n,") z rozwiązaniem x(",p,") = ", disc) 
    if details == True:
        print("")
        print("Zgodnie z twierdzeniem:")
        for i in range(fl):
            print("x =",x[i]," (mod", p_list[i],"^",e_list[i],")")

        print("\nWynik końcowy: x =",CRT_list(x,y))
    return CRT_list(x,y)

### PORÓWNANIE ALGORYTMÓW
import time
def Compare_algorithms(a,b,n):
    start_time = time.time()
    result = Brute_force_method(a, b, n)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Czas wykonania: Metoda przeliczania:",elapsed_time,"sekund\n")
    
    start_time = time.time()
    result = Baby_giant_step(a, b, n)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Czas wykonania: Algorytm małych i wielkich kroków:",elapsed_time,"sekund\n")
    
    start_time = time.time()
    result = Pohlig_Hellman_algorithm(a, b, n)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Czas wykonania: Algorytm Diffiego-Hellmana:",elapsed_time,"sekund")

# Wymiana klucza Diffiego-Hellmana

In [11]:
@interact
def interactive_diffie_hellman( p=input_box(19, label="Wartość p:"),
                                g=input_box(2, label="Wartość g:"),
                                a=input_box(7, label="Wartość a:"),
                                b=input_box(4, label="Wartość b:"),
                                execute=checkbox(False, label="Wykonaj")):
    if execute:
        result = Diffie_Hellman_key_exchange(p,g,a,b)


Interactive function <function interactive_diffie_hellman at 0x7fbd07fe71a0> with 5 widgets
  p: EvalText(value='19', description='Wartość p:', layout=Layout(max_width='81em'))
  g: EvalText(value='2', description='Wartość g:', layout=Layout(max_width='81em'))
  a: EvalText(value='7', description='Wartość a:', layout=Layout(max_width='81em'))
  b: EvalText(value='4', description='Wartość b:', layout=Layout(max_width='81em'))
  execute: Checkbox(value=False, description='Wykonaj')

# Generowanie klucza w systemie ElGamal'a

In [12]:
@interact
def interactive_ElGamal_public_key( p=input_box(113, label="Wartość p:"),
                                    g=input_box(3, label="Wartość g:"),
                                    k=input_box(37, label="Wartość k:"),
                                    execute=checkbox(False, label="Wykonaj")):
    if execute:
        result = ElGamal_public_key(p,g,k)

Interactive function <function interactive_ElGamal_public_key at 0x7fbd07fe7880> with 4 widgets
  p: EvalText(value='113', description='Wartość p:', layout=Layout(max_width='81em'))
  g: EvalText(value='3', description='Wartość g:', layout=Layout(max_width='81em'))
  k: EvalText(value='37', description='Wartość k:', layout=Layout(max_width='81em'))
  execute: Checkbox(value=False, description='Wykonaj')

# Algorytm szyfrowania ElGamal

In [13]:
@interact
def interactive_ElGamal_encryption_algorithm( message=input_box("SZYFR", label="Wiadomość:", type=str),
                                              users_key=input_box((43,3,22), label="Klucz:"),
                                              j=input_box(23, label="Wartość j:"),
                                              execute=checkbox(False, label="Szyfruj")):
    if execute:
        result = ElGamal_encryption_algorithm(message,users_key,j)

Interactive function <function interactive_ElGamal_encryption_algorithm at 0x7fbd07fe7420> with 4 widgets
  message: TransformText(value='SZYFR', description='Wiadomość:', layout=Layout(max_width='81em'))
  users_key: EvalText(value='(43, 3, 22)', description='Klucz:', layout=Layout(max_width='81em'))
  j: EvalText(value='23', description='Wartość j:', layout=Layout(max_width='81em'))
  execute: Checkbox(value=False, description='Szyfruj')

# Algorytm deszyfrowania ElGamal

In [14]:
@interact
def interactive_ElGamal_decryption_algorithm( C1=input_box(34, label="C1:", type=Integer),
                                              C2=input_box("[17, 26, 37, 31, 28]", label="C2:", type=str),
                                              secret_key=input_box(15, label="Klucz pryw:", type=Integer),
                                              public_key=input_box("(43,3,22)", label="Klucz publ:",type=str),
                                              execute=checkbox(False, label="Rozszyfruj")):
    if execute == True:
        cipher = (C1,eval(C2))
        public_key = eval(public_key)
        result = ElGamal_decryption_algorithm(cipher,secret_key,public_key)

Interactive function <function interactive_ElGamal_decryption_algorithm at 0x7fbd07fe6980> with 5 widgets
  C1: EvalText(value='34', description='C1:', layout=Layout(max_width='81em'))
  C2: TransformText(value='[17, 26, 37, 31, 28]', description='C2:', layout=Layout(max_width='81em'))
  secret_key: EvalText(value='15', description='Klucz pryw:', layout=Layout(max_width='81em'))
  public_key: TransformText(value='(43,3,22)', description='Klucz publ:', layout=Layout(max_width='81em'))
  execute: Checkbox(value=False, description='Rozszyfruj')

# Metoda przeliczania

In [15]:
@interact
def interactive_ElGamal_decryption_algorithm( a=input_box(2, label="Wartość a:", type=Integer),
                                              b=input_box(7, label="Wartość b", type=Integer),
                                              n=input_box(557, label="Wartość n", type=Integer),
                                              execute=checkbox(False, label="Oblicz")):
    if execute == True:
        result = Brute_force_method(a,b,n, details = True)

Interactive function <function interactive_ElGamal_decryption_algorithm at 0x7fbd07fe7d80> with 4 widgets
  a: EvalText(value='2', description='Wartość a:', layout=Layout(max_width='81em'))
  b: EvalText(value='7', description='Wartość b', layout=Layout(max_width='81em'))
  n: EvalText(value='557', description='Wartość n', layout=Layout(max_width='81em'))
  execute: Checkbox(value=False, description='Oblicz')

# Alogrytm małych i wielkich kroków

In [16]:
@interact
def interactive_baby_giant_step(a=input_box(2, label="Wartość a:", type=Integer),
                                b=input_box(7, label="Wartość b:", type=Integer),
                                n=input_box(557, label="Wartość n:", type=Integer),
                                details=checkbox(False, label="Szczegóły"),
                                sets_display=checkbox(False, label="Pokaż zbiory kroków"),
                                calculate=checkbox(False, label="Oblicz")):
    if calculate:
        if details:
            if sets_display == False:
                print("Szczegóły algorytmu:")
            else:
                print("Szczegóły algorytmu, wraz ze zbiorami kroków:")
        result = Baby_giant_step(a, b, n, details, sets_display)
        print("Wynik: x =",result)
        calculate = False
        interactive_baby_giant_step.calculate = False

Interactive function <function interactive_baby_giant_step at 0x7fbd07fe5300> with 6 widgets
  a: EvalText(value='2', description='Wartość a:', layout=Layout(max_width='81em'))
  b: EvalText(value='7', description='Wartość b:', layout=Layout(max_width='81em'))
  n: EvalText(value='557', description='Wartość n:', layout=Layout(max_width='81em'))
  details: Checkbox(value=False, description='Szczegóły')
  sets_display: Checkbox(value=False, description='Pokaż zbiory kroków')
  calculate: Checkbox(value=False, description='Oblicz')

# Alogrytm Pohliga-Hellmana

In [17]:
@interact
def interactive_pohlig_hellman(a=input_box(2, label="Wartość a:", type=Integer),
                               b=input_box(7, label="Wartość b:", type=Integer),
                               n=input_box(557, label="Wartość n:", type=Integer),
                               details=checkbox(False, label="Szczegóły"),
                               calculate=checkbox(False, label="Oblicz")):
    if calculate:
        if details:
            print("Szczegóły algorytmu:")
        result = Pohlig_Hellman_algorithm(a, b, n, details)
        print("Wynik: x =",result)
        calculate = False
        interactive_pohlig_hellman.calculate = False

Interactive function <function interactive_pohlig_hellman at 0x7fbd07fe6de0> with 5 widgets
  a: EvalText(value='2', description='Wartość a:', layout=Layout(max_width='81em'))
  b: EvalText(value='7', description='Wartość b:', layout=Layout(max_width='81em'))
  n: EvalText(value='557', description='Wartość n:', layout=Layout(max_width='81em'))
  details: Checkbox(value=False, description='Szczegóły')
  calculate: Checkbox(value=False, description='Oblicz')

# Porównanie czasu działania algorytmów

In [18]:
@interact
def interactive_compare_algorithms(a=input_box(2, label="Wartość a:", type=Integer),
                               b=input_box(7, label="Wartość b:", type=Integer),
                               n=input_box(1344546617, label="Wartość n:", type=Integer),
                               calculate=checkbox(False, label="Porównaj")):
    if calculate:
        result = Compare_algorithms(a,b,n)

Interactive function <function interactive_compare_algorithms at 0x7fbd07fe6a20> with 4 widgets
  a: EvalText(value='2', description='Wartość a:', layout=Layout(max_width='81em'))
  b: EvalText(value='7', description='Wartość b:', layout=Layout(max_width='81em'))
  n: EvalText(value='1344546617', description='Wartość n:', layout=Layout(max_width='81em'))
  calculate: Checkbox(value=False, description='Porównaj')

In [19]:
Compare_algorithms(2,7,1344546617)

Czas wykonania metody przeliczania: 147.0667700767517 sekund

Czas wykonania algorytmu małych i wielkich kroków: 33.723758935928345 sekund

Czas wykonania alogrytmu Diffiego-Hellmana: 0.008737564086914062 sekund
